In [1]:
# set environment variable (only for jupyter notebook)
import os
os.environ["UNFCCC_GHG_ROOT_PATH"] = "/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data"

In [2]:
import camelot
import primap2 as pm2
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from UNFCCC_GHG_data.helper import process_data_for_country
#warnings.filterwarnings("ignore")

from UNFCCC_GHG_data.helper import downloaded_data_path, extracted_data_path

In [3]:
# ###
# configuration
# ###

input_folder = downloaded_data_path / 'UNFCCC' / 'Guinea' / 'BUR1'
output_folder = extracted_data_path / 'UNFCCC' / 'Guinea'
if not output_folder.exists():
    output_folder.mkdir()

pdf_file = "Rapport_IGES-Guinee-BUR1_VF.pdf"
output_filename = 'GIN_BUR1_2023_'
compression = dict(zlib=True, complevel=9)


In [4]:
# primap2 format conversion
coords_cols = {
    "category": "category",
    "entity": "entity",
    "unit": "unit",
}

coords_defaults = {
    "source": "GIN-GHG-Inventory",
    "provenance": "measured",
    "area": "GIN",
    "scenario": "BUR1",
}

coords_terminologies = {
    "area": "ISO3",
    # TODO check if this is correct
    "category": "IPCC1996_2006_GIN_Inv",
    "scenario": "PRIMAP",
}

In [5]:
# gwp conversion is mentioned on page 20 in the report
gwp_to_use = "AR4GWP100"
coords_value_mapping = {
    'main' : {
        "unit": "PRIMAP1",
        "category": "PRIMAP1",
        "entity": {
            'HFCs': f"HFCS ({gwp_to_use})",
            'PFCs': f"PFCS ({gwp_to_use})",
            'SF6' : f"SF6 ({gwp_to_use})",
            'NMVOCs': 'NMVOC',
        }
    },
    'energy' : {
        "unit": "PRIMAP1",
        "category": "PRIMAP1",
        "entity": {
            'NMVOCs': 'NMVOC',
        }
    },
    'lulucf' : {
        "unit": "PRIMAP1",
        "category": "PRIMAP1",
        "entity": {
            'NMVOCs': 'NMVOC',
        }
    },
    'waste' : {
        "unit": "PRIMAP1",
        "category": "PRIMAP1",
        "entity": {
            'NMVOCs': 'NMVOC',
        }
    },
    'trend' : {
        "unit": "PRIMAP1",
        "category": "PRIMAP1",
        "entity": {
            'NMVOCs': 'NMVOC',
        }
    },
}

# TODO! Don't add MEMO if remove later 
filter_remove = {
    'f_memo': {"category": "MEMO"},
}

In [6]:
meta_data = {
    "references": "https://unfccc.int/BURs",
    "rights": "", # unknown
    "contact": "daniel-busch@climate-resource.de",
    "title": "Guinea. Biennial update report (BUR). BUR1",
    "comment": "Read fom pdf by Daniel Busch",
    "institution": "UNFCCC",
}

In [7]:
page_def_templates = {
    '110': {
        "area": ['36,718,589,87'],
        "cols": ['290,340,368,392,425,445,465,497,535,564'],
    },
    '111': {
        "area": ['36,736,587,107'],
        "cols": ['293,335,369,399,424,445,468,497,535,565'],
    },
    '112': {
        "area": ['35,733,588,106'],
        "cols": ['293,335,369,399,424,445,468,497,535,565'],
    },
    '113': {
        "area": ['35,733,588,106'],
        "cols": ['293,335,365,399,424,445,468,497,535,565'],
    },
    '131' : {
                "area": ['36,718,590,83'],
                "cols": ['293,332,370,406,442,480,516,554'],
            },
}

# for main table
header_inventory = ['Greenhouse gas source and sink categories',
                   'CO2', 'CH4', "N2O", 'HFCs', 'PFCs', 'SF6', 'NOx', 'CO', 'NMVOCs','SO2'
                   ]

unit_inventory = ['-'] + ['Gg'] * len(header_inventory) # one extra for the category columns
unit_inventory[4] = "GgCO2eq"
unit_inventory[5] = "GgCO2eq"
unit_inventory[6] = "GgCO2eq"

# for energy tables
header_energy = ['Greenhouse gas source and sink categories',
                   'CO2', 'CH4', "N2O", 'NOx', 'CO', 'NMVOCs','SO2'
                ]
unit_energy = ['-'] + ['Gg'] * len(header_energy) # one extra for the category columns

# for lulucf tables
header_lulucf = ['Greenhouse gas source and sink categories', 'CO2', 'CH4', "N2O", 'NOx', 'CO', 'NMVOCs']
unit_lulucf = ['-'] + ['Gg'] * (len(header_lulucf) - 1)

# for waste table
header_waste = ['Greenhouse gas source and sink categories', 'CO2', 'CH4', "N2O", 'NOx', 'CO', 'NMVOCs', 'SO2']
unit_waste = ['-'] + ['Gg'] * (len(header_waste) - 1)

# for trend table (unit is always Gg for this table)
header_trend = ['data1990', 'data1995', "data2000", 'data2005', 'data2010', 'data2015', 'data2018', 'data2019']


# define config dict
inv_conf = {
    'header': header_inventory,
    'unit': unit_inventory,
    'header_energy' : header_energy,
    'unit_energy' : unit_energy,
    'header_lulucf' : header_lulucf,
    'unit_lulucf' : unit_lulucf,
    'header_waste' : header_waste,
    'unit_waste' : unit_waste,
    'header_trend' : header_trend,
    'entity_row': 0,
    'unit_row': 1,
    'index_cols': "Greenhouse gas source and sink categories",
    'year': {'110' : 1990,
             '111' : 2000,
             '112' : 2010,
             '113' : 2019,
             '116' : 1990,
             '117' : 2000,
             '118' : 2010,
             '119' : 2019,
             '124' : 1990,
             '125' : 2000,
             '126' : 2010,
             '127' : 2019,
            },
    'header_long': ["orig_cat_name", "entity", "unit", "time", "data"],
    "cat_code_regexp" : r'^(?P<code>[a-zA-Z0-9\.]{1,11})[\s\.].*',
    "cat_codes_manual" : {
        'main' : {
            'Éléments pour mémoire': 'MEMO',
            'Soutes internationales': 'M.BK',
            '1.A.3.a.i - Aviation internationale (soutes internationales)': 'M.BK.A',
            '1.A.3.d.i - Navigation internationale (soutes internationales)' : 'M.BK.M',
            '1.A.5.c - Opérations multilatérales' : 'M.MULTIOP',
            'Total des émissions et absorptions nationales': "0",
            '2A5: Autre': '2A5',
        },
        'energy' : {
            'International Bunkers': 'MEMO',
            '1.A.3.a.i - Aviation internationale (soutes internationales)': 'M.BK.A',
            '1.A.3.d.i - Navigation internationale (soutes internationales)' : 'M.BK.M',
            '1.A.5.c - Opérations multilatérales' : 'M.MULTIOP',
        }
    },
}

## 1. Read main tables - pages 110, 111, 112, 113

In [8]:
pages = ['110', '111', '112', '113']
df_all_dict = {}
for page in pages:
    
    print("-"*45)
    print(f"Reading table from page {page}.")
    
    tables_inventory_original = camelot.read_pdf(
        str(input_folder / pdf_file),
        pages=page,
        table_areas=page_def_templates[page]["area"],
        columns=page_def_templates[page]["cols"],
        flavor="stream",
        split_text=True)
    
    print("Reading complete.")
    
    df_inventory = tables_inventory_original[0].df.copy()

    # move broken text in correct row (page 113 is fine)
    if page in ['110', '111', '112']:
        df_inventory.at[4, 0] = "1.A.1 - Industries énergétiques"
        df_inventory = df_inventory.drop(index=3)
        df_inventory.at[8, 0] = "1.A.4 - Autres secteurs"
        df_inventory = df_inventory.drop(index=7)

    # add header and unit
    df_header = pd.DataFrame([inv_conf["header"], inv_conf["unit"]])
    df_inventory = pd.concat([df_header, df_inventory], axis=0, join='outer').reset_index(drop=True)
    df_inventory = pm2.pm2io.nir_add_unit_information(df_inventory,
                                                  unit_row=inv_conf["unit_row"],
                                                  entity_row=inv_conf["entity_row"],
                                                  regexp_entity=".*",
                                                  regexp_unit=".*",
                                                  default_unit="Gg")
    
    print("Added unit information.")
    
    # set index
    df_inventory = df_inventory.set_index(inv_conf["index_cols"])

    # convert to long format
    df_inventory_long = pm2.pm2io.nir_convert_df_to_long(df_inventory, inv_conf["year"][page],
                                                     inv_conf["header_long"])

    # extract category from tuple
    df_inventory_long["orig_cat_name"] = df_inventory_long["orig_cat_name"].str[0] 

    # prep for conversion to PM2 IF and native format
    # make a copy of the categories row
    df_inventory_long["category"] = df_inventory_long["orig_cat_name"]

    # replace cat names by codes in col "category"
    # first the manual replacements
    # TODO: move this to config section
#    inv_conf["cat_codes_manual"]['main'] = {
#            'Éléments pour mémoire': 'MEMO',
#            'Soutes internationales': 'M.BK',
#            '1.A.3.a.i - Aviation internationale (soutes internationales)': 'M.BK.A',
#            '1.A.3.d.i - Navigation internationale (soutes internationales)' : 'M.BK.M',
#            '1.A.5.c - Opérations multilatérales' : 'M.MULTIOP',
#            'Total des émissions et absorptions nationales': "0",
#            '2A5: Autre': '2A5', 
#        }
    df_inventory_long["category"] = \
        df_inventory_long["category"].replace(inv_conf["cat_codes_manual"]['main'])  

    df_inventory_long["category"] = df_inventory_long["category"].str.replace(".", "")
    
    # then the regex replacements
    repl = lambda m: m.group('code')
    df_inventory_long["category"] = \
        df_inventory_long["category"].str.replace(inv_conf["cat_code_regexp"], repl,
                                              regex=True)

    df_inventory_long = df_inventory_long.reset_index(drop=True)


    
    df_inventory_long["data"] = df_inventory_long["data"].str.replace(",", ".")
    df_inventory_long["data"] = df_inventory_long["data"].str.replace("NE1", "NE")

    # make sure all col headers are str
    df_inventory_long.columns = df_inventory_long.columns.map(str)
    df_inventory_long = df_inventory_long.drop(columns=["orig_cat_name"])
    
    df_all_dict[page] = df_inventory_long

df_all = pd.concat([df_all_dict['110'], df_all_dict['111'], df_all_dict['112'], df_all_dict['113']],
                      axis=0,
                      join='outer').reset_index(drop=True)

print("Converting to interchange format.")
df_all_IF = pm2.pm2io.convert_long_dataframe_if(
    df_all,
    coords_cols=coords_cols,
    #add_coords_cols=add_coords_cols,
    coords_defaults=coords_defaults,
    coords_terminologies=coords_terminologies,
    coords_value_mapping=coords_value_mapping['main'],
    #coords_value_filling=coords_value_filling,
    filter_remove=filter_remove,
    #filter_keep=filter_keep,
    meta_data=meta_data,
    convert_str=True,
    time_format="%Y",
    )



---------------------------------------------
Reading table from page 110.
Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 111.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 112.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 113.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
Converting to interchange format.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


In [9]:
# There are different values for the same categories in the main and lulucf table
# it looks like in the main table they put the value from 1990 again for 2019 again
# it's unlikely that the value is exactly the same for 1990 and 2019
# so I assume the other one is correct
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3") & (df_all_IF["entity"] == "CO") , "2019"] = 27.406
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.C") & (df_all_IF["entity"] == "CO") , "2019"] = 27.406
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.C.1") & (df_all_IF["entity"] == "CO") , "2019"] = 27.406

# Values for category 3 and N2O are identical for 1990 and 2019
# The sum of the sub-categories does not equal the value of the parent category
# The value  in the lulucf table should therefore be the correct one
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3") & (df_all_IF["entity"] == "N2O") , "1990"] = 2.190

# Values for category 3 and NOx are identical for 1990 and 2019
# Replacing the duplicate value with the value from the lulucf table
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3") & (df_all_IF["entity"] == "NOx") , "2019"] = 1.644
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.C") & (df_all_IF["entity"] == "NOx") , "2019"] = 1.644
df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.C.1") & (df_all_IF["entity"] == "NOx") , "2019"] = 1.644

In [10]:
#df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "1.A.3.a.i") & (df_all_IF["entity"] == "N2O"), "1990"].values
#df_all_IF.loc[(df_all_IF["category (IPCC1996_2006_GIN_Inv)"] == "1.A.3"), '2010'].values

In [12]:
### Test individual values from the tables ###
# TODO and note: this function is work in progress
# Use assert statements and print error message
# with category, entity, year, expected value and actual value

### Test individual values from the tables ###
def assert_individual_value(
    df,
    category_column,
    entity_column,
    category,
    entity,
    year,
    expected_value
):
    arr = df.loc[(df[category_column] == category) & (df[entity_column] == entity), year].values
    print(arr)
    if len(arr) > 1:
        print(f"More than one value found for {category}, {entity}, {year}!")

    # TODO: It looks like this will be true when the value equals 0
    if not arr.size > 0:
        print((f"No value found for {category}, {entity}, {year}!"))
            
    if not arr[0] == expected_value:
        print(f"Expected value {expected_value}, actual value is {arr[0]}")

    if arr[0] == expected_value:
        print("Value matches expected value.")

    return


test_cases = {
    "1" : {
        "category" : "1.A.1",
        'entity' : "CO2",
        "year" : "2010",
        "expected_value" : 422.474,
    },
    "2" : {
        "category" : "2",
        'entity' : "SO2",
        "year" : "1990",
        "expected_value" : 0.097,
    },
    "3" : {
        "category" : "M.BK.A",
        'entity' : "N2O",
        "year" : "2000",
        "expected_value" : 6e-5,
    },
    '4' : {
        "category" : "2.H.2",
        'entity' : "NMVOC",
        "year" : "2019",
        "expected_value" : 2.506,
    },
    '5' : {
        "category" : "1.A.1",
        'entity' : "CH4",
        "year" : "2019",
        "expected_value" : 0.0011,
    }
}

for key in test_cases.keys():
    print("-"*50)
    print(f"Testing combination {test_cases[key]['category']}, {test_cases[key]['entity']}, {test_cases[key]['year']}.")
    assert_individual_value(
                    df = df_all_IF,
                    category_column = "category (IPCC1996_2006_GIN_Inv)",
                    entity_column = "entity",
                    category = test_cases[key]["category"],
                    entity = test_cases[key]["entity"],
                    year = test_cases[key]["year"],
                    expected_value = test_cases[key]["expected_value"])

--------------------------------------------------
Testing combination 1.A.1, CO2, 2010.
[422.474]
Value matches expected value.
--------------------------------------------------
Testing combination 2, SO2, 1990.
[0.097]
Value matches expected value.
--------------------------------------------------
Testing combination M.BK.A, N2O, 2000.
[6.e-05]
Value matches expected value.
--------------------------------------------------
Testing combination 2.H.2, NMVOC, 2019.
[2.506]
Value matches expected value.
--------------------------------------------------
Testing combination 1.A.1, CH4, 2019.
[0.0011]
Value matches expected value.


In [ ]:
### check data for errors ###
# print a few things to see if it looks "normal"
for c in df_all_IF.columns:
    print('-'*50)
    print(f"Unique values in column {c}")
    print(df_all_IF[c].unique())

In [13]:
### convert to primap2 format ###
data_pm2_main = pm2.pm2io.from_interchange_format(df_all_IF)

2024-03-29 12:27:31.256 | DEBUG    | primap2.pm2io._interchange_format:from_interchange_format:320 - Expected array shapes: [[1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78], [1, 1, 1, 1, 10, 78]], resulting in size 7,800.
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/_data_format.py:481: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  all_dims = set(ds.dims.keys())
2024-03-29 12:27:31.452 | INFO     | primap2._data_format:ensure_valid_attributes:292 - Reference information is not a DOI: 'https://unfccc.int/BURs'


## 2. Read in sector tables for energy - pages 116, 117, 118, 119

In [14]:
pages = ['116', '117', '118', '119']
df_energy_dict = {}
for page in pages:
    print("-"*45)
    print(f"Reading table from page {page}.")
    
    tables_inventory_original = camelot.read_pdf(
        str(input_folder / pdf_file),
        pages=page,
        flavor="lattice",
        split_text=True
        )
    
    print("Reading complete.")

    # cut last two lines of second table to ignore additional information regarding biomass for energy production 
    df_energy_year = pd.concat([tables_inventory_original[0].df[2:],
                                tables_inventory_original[1].df[3:-2]],
                                axis=0,
                                join='outer').reset_index(drop=True)

    
    # drop duplicate lines - 1.A.3.d.i / 1.A.3.a.i / 1.A.5.c
    # TODO: better to find the index of the line and then drop it by the index
    df_energy_year = df_energy_year.drop(index=[27, 32, 50])  
    
    # add header and unit
    df_header = pd.DataFrame([inv_conf["header_energy"], inv_conf["unit_energy"]])

    df_energy_year = pd.concat([df_header, df_energy_year], axis=0, join='outer').reset_index(drop=True)
    
    df_energy_year = pm2.pm2io.nir_add_unit_information(df_energy_year,
                                                  unit_row=inv_conf["unit_row"],
                                                  entity_row=inv_conf["entity_row"],
                                                  regexp_entity=".*",
                                                  regexp_unit=".*",
                                                  default_unit="Gg")
    
    print("Added unit information.")
    # set index
    df_energy_year = df_energy_year.set_index(inv_conf["index_cols"])

    # convert to long format
    df_energy_year_long = pm2.pm2io.nir_convert_df_to_long(df_energy_year, inv_conf["year"][page],
                                                     inv_conf["header_long"])
    
    # extract from tuple
    df_energy_year_long["orig_cat_name"] = df_energy_year_long["orig_cat_name"].str[0] 

    # prep for conversion to PM2 IF and native format
    # make a copy of the categories row
    df_energy_year_long["category"] = df_energy_year_long["orig_cat_name"]

    # replace individual categories
    # TODO: move to config section
    #inv_conf["cat_codes_manual"]['energy'] = {
    #        'International Bunkers': 'MEMO',
    #        '1.A.3.a.i - Aviation internationale (soutes internationales)': 'M.BK.A',
    #        '1.A.3.d.i - Navigation internationale (soutes internationales)' : 'M.BK.M',
    #        '1.A.5.c - Opérations multilatérales' : 'M.MULTIOP',
    #    }

    # replace cat names by codes in col "category"
    # first the manual replacements
    df_energy_year_long["category"] = df_energy_year_long["category"].str.replace('\n' ,'')
    df_energy_year_long["category"] = \
        df_energy_year_long["category"].replace(inv_conf["cat_codes_manual"]['energy'])

    df_energy_year_long["category"] = df_energy_year_long["category"].str.replace(".", "")
    
    #inv_conf["cat_code_regexp"] = r'^(?P<code>[a-zA-Z0-9\.]{1,11})[\s\.].*'

    # then the regex replacements
    repl = lambda m: m.group('code')
    df_energy_year_long["category"] = \
        df_energy_year_long["category"].str.replace(inv_conf["cat_code_regexp"], repl,
                                              regex=True)

    df_energy_year_long = df_energy_year_long.reset_index(drop=True)

    
    df_energy_year_long["data"] = df_energy_year_long["data"].str.replace(",", ".")
    df_energy_year_long["data"] = df_energy_year_long["data"].str.replace("NE1", "NE")

    # make sure all col headers are str
    df_energy_year_long.columns = df_energy_year_long.columns.map(str)
    df_energy_year_long = df_energy_year_long.drop(columns=["orig_cat_name"])
    
    df_energy_dict[page] = df_energy_year_long

df_energy = pd.concat([df_energy_dict['116'], df_energy_dict['117'], df_energy_dict['118'], df_energy_dict['119']],
                      axis=0,
                      join='outer').reset_index(drop=True)

print("Converting to interchange format.")
df_energy_IF = pm2.pm2io.convert_long_dataframe_if(
    df_energy,
    coords_cols=coords_cols,
    #add_coords_cols=add_coords_cols,
    coords_defaults=coords_defaults,
    coords_terminologies=coords_terminologies,
    coords_value_mapping=coords_value_mapping['energy'],
    #coords_value_filling=coords_value_filling,
    filter_remove=filter_remove,
    #filter_keep=filter_keep,
    meta_data=meta_data,
    convert_str=True,
    time_format="%Y",
    )
    
df_energy_IF

---------------------------------------------
Reading table from page 116.
Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 117.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 118.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 119.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
Converting to interchange format.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


,source,scenario (PRIMAP),provenance,area (ISO3),entity,unit,category (IPCC1996_2006_GIN_Inv),1990,2000,2010,2019
0,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1,6.465,6.489,4.849,5.821
1,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A,6.465,6.489,4.849,5.821
2,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A.1,0.032,0.024,0.016,0.001
3,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A.1.a,0.032,0.024,0.016,0.001
4,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A.1.a.i,0.032,0.024,0.016,0.001
...,...,...,...,...,...,...,...,...,...,...,...
373,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,1.A.5.b.iii,NaN,NaN,NaN,NaN
374,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,1.A.5.c,NaN,NaN,NaN,NaN
375,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,1.B,NaN,NaN,NaN,NaN
376,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,M.BK.M,NaN,NaN,NaN,NaN


In [15]:
df_energy_IF['entity'].unique()

array(['CH4', 'CO', 'CO2', 'N2O', 'NMVOC', 'NOx', 'SO2'], dtype=object)

In [17]:
test_cases = {
    "1" : {
        "category" : "1.A.2.k",
        'entity' : "CH4",
        "year" : "1990",
        "expected_value" : 3e-05,
    },
    "2" : {
        "category" : "1.A.4.c.i",
        'entity' : "CO",
        "year" : "1990",
        "expected_value" : 0.0016,
    },
    "3" : {
        "category" : "1.A.3.a.i",
        'entity' : "NMVOC",
        "year" : "2000",
        "expected_value" : 0.0002,
    },
    '4' : {
        "category" : "1",
        'entity' : "SO2",
        "year" : "2010",
        "expected_value" : 0,
    },
    '5' : {
        "category" : "1.A.2.k",
        'entity' : "N2O",
        "year" : "2019",
        "expected_value" : 7e-06,
    }
}

for key in test_cases.keys():
    print("-"*50)
    print(f"Testing combination {test_cases[key]['category']}, {test_cases[key]['entity']}, {test_cases[key]['year']}.")
    assert_individual_value(
                    df = df_energy_IF,
                    category_column = "category (IPCC1996_2006_GIN_Inv)",
                    entity_column = "entity",
                    category = test_cases[key]["category"],
                    entity = test_cases[key]["entity"],
                    year = test_cases[key]["year"],
                    expected_value = test_cases[key]["expected_value"])

--------------------------------------------------
Testing combination 1.A.2.k, CH4, 1990.
[3.e-05]
Value matches expected value.
--------------------------------------------------
Testing combination 1.A.4.c.i, CO, 1990.
[0.0016]
Value matches expected value.
--------------------------------------------------
Testing combination 1.A.3.a.i, NMVOC, 2000.
[0.0002]
Value matches expected value.
--------------------------------------------------
Testing combination 1, SO2, 2010.
[0.]
Value matches expected value.
--------------------------------------------------
Testing combination 1.A.2.k, N2O, 2019.
[7.e-06]
Value matches expected value.


In [18]:
### convert to primap2 format ###
data_pm2_energy = pm2.pm2io.from_interchange_format(df_energy_IF)

2024-03-29 12:28:16.650 | DEBUG    | primap2.pm2io._interchange_format:from_interchange_format:320 - Expected array shapes: [[1, 1, 1, 1, 7, 54], [1, 1, 1, 1, 7, 54], [1, 1, 1, 1, 7, 54], [1, 1, 1, 1, 7, 54], [1, 1, 1, 1, 7, 54], [1, 1, 1, 1, 7, 54], [1, 1, 1, 1, 7, 54]], resulting in size 2,646.
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/_data_format.py:481: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  all_dims = set(ds.dims.keys())
2024-03-29 12:28:16.770 | INFO     | primap2._data_format:ensure_valid_attributes:292 - Reference information is not a DOI: 'https://unfccc.int/BURs'


# 3. Read in LULUCF table - pages 124, 125, 126, 127

In [19]:
pages = ['124', '125', '126', '127']
df_lulucf_dict = {}
for page in pages:
    print("-"*45)
    print(f"Reading table from page {page}.")
    
    tables_inventory_original = camelot.read_pdf(
    str(input_folder / pdf_file),
    pages=page,
    flavor="lattice",
    split_text=True
    )
    print("Reading complete.")

    if page == '127':
        # table on page 127 has one extra row at the top
        # and one extra category 3.A.1.j
        df_lulucf_year = tables_inventory_original[0].df[3:]
        # rename duplicate categories in tables
        # TODO move to config section
        replace_categories = [(19, "3.A.2.a.i - Vaches laitières"),
                              (20, "3.A.2.a.ii - Autres bovins"),
                              (21, "3.A.2.b - Buffle"),
                              (22, "3.A.2.c - Ovins"),
                              (23, "3.A.2.d - Caprins"),
                              (24, "3.A.2.e - Chameaux"),
                              (25, "3.A.2.f - Chevaux"),
                              (26, "3.A.2.g - Mules et ânes"),
                              (27, "3.A.2.h - Porcins"),
                              (28, "3.A.2.i - Volailles"),
                              (29, "3.A.2.j - Autres (préciser)"),]
        for index, category_name in  replace_categories:
            df_lulucf_year.at[index, 0] = category_name
    else:
        # cut first two lines
        df_lulucf_year = tables_inventory_original[0].df[2:] 

        # TODO move to config section
        replace_categories = [(17, "3.A.2.a.i - Vaches laitières"),
                              (18, "3.A.2.a.ii - Autres bovins"),
                              (19, "3.A.2.b - Buffle"),
                              (20, "3.A.2.c - Ovins"),
                              (21, "3.A.2.d - Caprins"),
                              (22, "3.A.2.e - Chameaux"),
                              (23, "3.A.2.f - Chevaux"),
                              (24, "3.A.2.g - Mules et ânes"),
                              (25, "3.A.2.h - Porcins"),
                              (26, "3.A.2.i - Volailles"),]
        for index, category_name in  replace_categories:
            df_lulucf_year.at[index, 0] = category_name
    
    # add header and unit
    df_header = pd.DataFrame([inv_conf["header_lulucf"], inv_conf["unit_lulucf"]])

    df_lulucf_year = pd.concat([df_header, df_lulucf_year], axis=0, join='outer').reset_index(drop=True)

    df_lulucf_year = pm2.pm2io.nir_add_unit_information(df_lulucf_year,
                                                  unit_row=inv_conf["unit_row"],
                                                  entity_row=inv_conf["entity_row"],
                                                  regexp_entity=".*",
                                                  regexp_unit=".*",
                                                  default_unit="Gg")

    print("Added unit information.")
    
    # set index
    df_lulucf_year = df_lulucf_year.set_index(inv_conf["index_cols"])

    # convert to long format
    df_lulucf_year_long = pm2.pm2io.nir_convert_df_to_long(df_lulucf_year, inv_conf["year"][page],
                                                     inv_conf["header_long"])
    
    df_lulucf_year_long["orig_cat_name"] = df_lulucf_year_long["orig_cat_name"].str[0] # extract from tuple

    # prep for conversion to PM2 IF and native format
    # make a copy of the categories row
    df_lulucf_year_long["category"] = df_lulucf_year_long["orig_cat_name"]
   
    # regex replacements
    repl = lambda m: m.group('code')
    df_lulucf_year_long["category"] = \
        df_lulucf_year_long["category"].str.replace(inv_conf["cat_code_regexp"], repl,
                                              regex=True)
    
    df_lulucf_year_long = df_lulucf_year_long.reset_index(drop=True)
    
    df_lulucf_year_long["data"] = df_lulucf_year_long["data"].str.replace(",", ".")
    df_lulucf_year_long["data"] = df_lulucf_year_long["data"].str.replace("NE1", "NE")

    # make sure all col headers are str
    df_lulucf_year_long.columns = df_lulucf_year_long.columns.map(str)
    df_lulucf_year_long = df_lulucf_year_long.drop(columns=["orig_cat_name"])
    
    df_lulucf_dict[page] = df_lulucf_year_long

df_lulucf = pd.concat([df_lulucf_dict['124'], df_lulucf_dict['125'], df_lulucf_dict['126'], df_lulucf_dict['127']],
                      axis=0,
                      join='outer').reset_index(drop=True)

print("Converting to interchange format.")
df_lulucf_IF = pm2.pm2io.convert_long_dataframe_if(
    df_lulucf,
    coords_cols=coords_cols,
    #add_coords_cols=add_coords_cols,
    coords_defaults=coords_defaults,
    coords_terminologies=coords_terminologies,
    coords_value_mapping=coords_value_mapping['lulucf'],
    #coords_value_filling=coords_value_filling,
    filter_remove=filter_remove,
    #filter_keep=filter_keep,
    meta_data=meta_data,
    convert_str=True,
    time_format="%Y",
    )
    
df_lulucf_IF

---------------------------------------------
Reading table from page 124.
Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 125.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 126.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
---------------------------------------------
Reading table from page 127.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


Reading complete.
Added unit information.
Converting to interchange format.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()


,source,scenario (PRIMAP),provenance,area (ISO3),entity,unit,category (IPCC1996_2006_GIN_Inv),1990,2000,2010,2019
0,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,3,56.987,110.568,187.617,299.503
1,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,3.A,55.634,107.911,186.769,298.533
2,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,3.A.1,53.796,104.298,180.454,288.239
3,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,3.A.1.a,49.050,94.967,161.753,256.319
4,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,3.A.1.a.i,10.488,17.802,27.091,31.905
...,...,...,...,...,...,...,...,...,...,...,...
469,GIN-GHG-Inventory,BUR1,measured,GIN,NOx,Gg NOx / yr,3.C.7,0.000,0.000,0.000,0.000
470,GIN-GHG-Inventory,BUR1,measured,GIN,NOx,Gg NOx / yr,3.C.8,0.000,0.000,0.000,0.000
471,GIN-GHG-Inventory,BUR1,measured,GIN,NOx,Gg NOx / yr,3.D,0.000,0.000,0.000,0.000
472,GIN-GHG-Inventory,BUR1,measured,GIN,NOx,Gg NOx / yr,3.D.1,0.000,0.000,0.000,0.000


In [20]:
df_lulucf_IF['entity'].unique()

array(['CH4', 'CO', 'CO2', 'N2O', 'NMVOC', 'NOx'], dtype=object)

In [21]:
### convert to primap2 format ###
data_pm2_lulucf = pm2.pm2io.from_interchange_format(df_lulucf_IF)

2024-03-29 12:29:29.030 | DEBUG    | primap2.pm2io._interchange_format:from_interchange_format:320 - Expected array shapes: [[1, 1, 1, 1, 6, 79], [1, 1, 1, 1, 6, 79], [1, 1, 1, 1, 6, 79], [1, 1, 1, 1, 6, 79], [1, 1, 1, 1, 6, 79], [1, 1, 1, 1, 6, 79]], resulting in size 2,844.
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/_data_format.py:481: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  all_dims = set(ds.dims.keys())
2024-03-29 12:29:29.156 | INFO     | primap2._data_format:ensure_valid_attributes:292 - Reference information is not a DOI: 'https://unfccc.int/BURs'


# 3. Read in Waste tables - pages 128, 130

In [22]:
# There are three tables for three years on page 128
# and another tabel on page 130

# read three tables
page = '128'
tables_inventory_original_128 = camelot.read_pdf(
    str(input_folder / pdf_file),
    pages=page,
    flavor="lattice",
    split_text=True
)

# read last table
page = '130'
tables_inventory_original_130 = camelot.read_pdf(
    str(input_folder / pdf_file),
    pages=page,
    flavor="lattice",
    split_text=True
)

# save to dict
df_waste_years = {
    '1990' : tables_inventory_original_128[0].df,
    '2000' : tables_inventory_original_128[1].df,
    '2010' : tables_inventory_original_128[2].df,
    '2019' : tables_inventory_original_130[0].df,
}


In [23]:
df_waste_dict = {}
for year in df_waste_years.keys():
    print("-"*45)
    print(f"Processing table for {year}.")

    df_waste_year = df_waste_years[year][2:]
    
    # add header and unit
    df_header = pd.DataFrame([inv_conf["header_waste"], inv_conf["unit_waste"]])

    df_waste_year = pd.concat([df_header, df_waste_year], axis=0, join='outer').reset_index(drop=True)

    df_waste_year = pm2.pm2io.nir_add_unit_information(df_waste_year,
                                                  unit_row=inv_conf["unit_row"],
                                                  entity_row=inv_conf["entity_row"],
                                                  regexp_entity=".*",
                                                  regexp_unit=".*",
                                                  default_unit="Gg")

    print("Added unit information.")
    
    # set index
    df_waste_year = df_waste_year.set_index(inv_conf["index_cols"])

    # convert to long format
    df_waste_year_long = pm2.pm2io.nir_convert_df_to_long(df_waste_year, year,
                                                     inv_conf["header_long"])
    
    df_waste_year_long["orig_cat_name"] = df_waste_year_long["orig_cat_name"].str[0]

    # prep for conversion to PM2 IF and native format
    # make a copy of the categories row
    df_waste_year_long["category"] = df_waste_year_long["orig_cat_name"]

    # regex replacements
    repl = lambda m: m.group('code')
    df_waste_year_long["category"] = \
        df_waste_year_long["category"].str.replace(inv_conf["cat_code_regexp"], repl,
                                              regex=True)
    
    df_waste_year_long = df_waste_year_long.reset_index(drop=True)

    df_waste_year_long["category"] = df_waste_year_long["category"].str.replace(".", "")
    df_waste_year_long["data"] = df_waste_year_long["data"].str.replace(",", ".")
    df_waste_year_long["data"] = df_waste_year_long["data"].str.replace("NE1", "NE")

    # make sure all col headers are str
    df_waste_year_long.columns = df_waste_year_long.columns.map(str)
    df_waste_year_long = df_waste_year_long.drop(columns=["orig_cat_name"])
    
    df_waste_dict[year] = df_waste_year_long

df_waste = pd.concat([df_waste_dict['1990'], df_waste_dict['2000'], df_waste_dict['2010'], df_waste_dict['2019']],
                      axis=0,
                      join='outer').reset_index(drop=True)

print("Converting to interchange format.")
df_waste_IF = pm2.pm2io.convert_long_dataframe_if(
    df_waste,
    coords_cols=coords_cols,
    #add_coords_cols=add_coords_cols,
    coords_defaults=coords_defaults,
    coords_terminologies=coords_terminologies,
    coords_value_mapping=coords_value_mapping['waste'],
    #coords_value_filling=coords_value_filling,
    filter_remove=filter_remove,
    #filter_keep=filter_keep,
    meta_data=meta_data,
    convert_str=True,
    time_format="%Y",
    )
    
df_waste_IF

---------------------------------------------
Processing table for 1990.
Added unit information.
---------------------------------------------
Processing table for 2000.
Added unit information.
---------------------------------------------
Processing table for 2010.
Added unit information.
---------------------------------------------
Processing table for 2019.
Added unit information.
Converting to interchange format.


/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_inventory_reading.py:167: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df_nir.stack([0, 1], dropna=True).to_frame()
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/pm2io/_GHG_i

,source,scenario (PRIMAP),provenance,area (ISO3),entity,unit,category (IPCC1996_2006_GIN_Inv),1990,2000,2010,2019
0,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,4,1.750,2.925,4.534,6.665
1,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,4.A,1.029,2.054,3.323,5.170
2,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,4.A.1,NaN,NaN,NaN,NaN
3,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,4.A.2,NaN,NaN,NaN,NaN
4,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,4.A.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
86,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,4.C.2,0.000,0.000,0.000,0.000
87,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,4.D,0.000,0.000,0.000,0.000
88,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,4.D.1,0.000,0.000,0.000,0.000
89,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,4.D.2,0.000,0.000,0.000,0.000


In [24]:
### convert to primap2 format ###
data_pm2_waste = pm2.pm2io.from_interchange_format(df_waste_IF)

2024-03-29 12:29:37.042 | DEBUG    | primap2.pm2io._interchange_format:from_interchange_format:320 - Expected array shapes: [[1, 1, 1, 1, 7, 13], [1, 1, 1, 1, 7, 13], [1, 1, 1, 1, 7, 13], [1, 1, 1, 1, 7, 13], [1, 1, 1, 1, 7, 13], [1, 1, 1, 1, 7, 13], [1, 1, 1, 1, 7, 13]], resulting in size 637.
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/_data_format.py:481: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  all_dims = set(ds.dims.keys())
2024-03-29 12:29:37.102 | INFO     | primap2._data_format:ensure_valid_attributes:292 - Reference information is not a DOI: 'https://unfccc.int/BURs'


# 4. Read in trend tables - pages 131 - 137

In [25]:
#%matplotlib widget 
#camelot.plot(tables_inventory_original[0], kind='text')

df_main_dict = {}
pages = ['131', '132', '133', '134', '135', '136', '137']
entities = ['CO2', 'CH4', 'N2O', 'NOx', 'CO', 'NMVOCs', 'SO2']

# for this set of tables every page is a different entity
for page, entity in zip(pages, entities):

    print("-"*45)
    print(f"Reading table for page {page} and entity {entity}.")
    
    # first table needs to be read in with flavor="stream"
    # flavor="lattice" raises an error, maybe camelot issue
    # see https://github.com/atlanhq/camelot/issues/306
    # or because characters in first row almost reach
    # the table grid    
    if page == '131':
        tables_inventory_original = camelot.read_pdf(
            str(input_folder / pdf_file),
            pages=page,
            table_areas=page_def_templates[page]["area"],
            columns=page_def_templates[page]["cols"],
            flavor="stream",
            split_text=True
        )
        
        df_trend_entity = tables_inventory_original[0].df[1:]

        # these rows are different to the main table and don't make sense
        row_to_delete = df_trend_entity.index[df_trend_entity[0] == '3.D - Autres'][0]
        df_trend_entity = df_trend_entity.drop(index = row_to_delete)

        row_to_delete = df_trend_entity.index[df_trend_entity[0] == '3.D.1 - Produits ligneux récoltés'][0]
        df_trend_entity = df_trend_entity.drop(index = row_to_delete)

        row_to_delete = df_trend_entity.index[df_trend_entity[0] == '3.D.2 - Autres (veuillez spécifier)'][0]
        df_trend_entity = df_trend_entity.drop(index = row_to_delete)
   
    else:
        tables_inventory_original = camelot.read_pdf(
            str(input_folder / pdf_file),
            pages=page,
            flavor="lattice",
            split_text=True)
        df_trend_entity = tables_inventory_original[0].df[3:]

    print(f"Reading complete.")

    # add columns
    # 'data' prefix is needed for pd.wide_to_long() later
    columns_years = ['data1990', 'data1995', "data2000", 'data2005', 'data2010', 'data2015', 'data2018', 'data2019']
    df_trend_entity.columns = ['orig_cat_name'] + columns_years
    
    # unit is always Gg
    df_trend_entity.loc[:, 'unit'] = 'Gg'
    
    # only one entity per table
    df_trend_entity.loc[:,'entity'] = entity
    
    df_trend_entity.loc[:, "category"] = df_trend_entity["orig_cat_name"]

    # delete rows that are just a headline or empty
    #row_to_delete = df_trend_entity.index[df_trend_entity['category'] == 'Éléments pour mémoire'][0]
    #df_trend_entity = df_trend_entity.drop(index = row_to_delete)

    # in the first table there is no empty line
    if page != '131':
        row_to_delete = df_trend_entity.index[df_trend_entity['category'] == ''][0]
        df_trend_entity = df_trend_entity.drop(index = row_to_delete)
    
    inv_conf["cat_code_regexp"] = r'^(?P<code>[a-zA-Z0-9\.]{1,11})[\s\.].*'

    df_trend_entity["category"] = df_trend_entity["category"].replace(
        {
         'Total des émissions et absorptions nationales': "0",
         '2A5: Autre' : '2A5',
         'Éléments pour mémoire': 'MEMO',
         'Soutes internationales' : 'M.BK',
         '1.A.3.a.i - Aviation internationale (soutes internationales)' : 'M.BK.A',
         '1.A.3.d.i - Navigation internationale (soutes internationales)' : 'M.BK.M',
         '1.A.5.c - Opérations multilatérales' : 'M.MULTIOP',
        })

    df_trend_entity.loc[:, "category"] = df_trend_entity["category"].str.replace(".", "")
    df_trend_entity.loc[:, "category"] = df_trend_entity["category"].str.replace("\n", "")
    
    
    repl = lambda m: m.group('code')
    df_trend_entity.loc[:, "category"] = \
        df_trend_entity["category"].str.replace(inv_conf["cat_code_regexp"], repl,
                                              regex=True)
    
    df_trend_entity = df_trend_entity.reset_index(drop=True)
    
    print(f"Created category codes.")

    if entity == 'CO':
        df_trend_entity = df_trend_entity.drop(columns=['data2010', 'data2000', 'data2019'])
        columns_years = ['data1990', 'data1995', 'data2005', 'data2015', 'data2018']

    for year in columns_years:
        df_trend_entity.loc[:, year] = df_trend_entity[year].str.replace(",", ".")
        df_trend_entity.loc[:, year] = df_trend_entity[year].str.replace("NE1", "NE")
    
    # make sure all col headers are str
    df_trend_entity.columns = df_trend_entity.columns.map(str)
    
    df_trend_entity = df_trend_entity.drop(columns=["orig_cat_name"])

    # TODO wide in IF gibt es convert_wide_dataframe_if
    df_trend_entity_long = pd.wide_to_long(df_trend_entity, stubnames='data',  i='category', j='time')
    
    print(f"Converted to long format.")
    
    df_trend_entity_long = df_trend_entity_long.reset_index()
    
    df_main_dict[page] =  df_trend_entity_long

print("Converting to interchange format.")

---------------------------------------------
Reading table for page 131 and entity CO2.
Reading complete.
Created category codes.
Converted to long format.
---------------------------------------------
Reading table for page 132 and entity CH4.
Reading complete.
Created category codes.
Converted to long format.
---------------------------------------------
Reading table for page 133 and entity N2O.


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:, 'unit'] = 'Gg'
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:,'entity'] = entity
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Reading complete.
Created category codes.
Converted to long format.
---------------------------------------------
Reading table for page 134 and entity NOx.


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:, 'unit'] = 'Gg'
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:,'entity'] = entity
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Reading complete.
Created category codes.
Converted to long format.
---------------------------------------------
Reading table for page 135 and entity CO.


...

/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:, 'unit'] = 'Gg'
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:,'entity'] = entity
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Reading complete.
Created category codes.
Converted to long format.
---------------------------------------------
Reading table for page 137 and entity SO2.


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:, 'unit'] = 'Gg'
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:,'entity'] = entity
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Reading complete.
Created category codes.
Converted to long format.
Converting to interchange format.


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:, 'unit'] = 'Gg'
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trend_entity.loc[:,'entity'] = entity
/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/4093200453.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [26]:
df_trend_all = pd.concat([df_main_dict['131'],
                          df_main_dict['132'],
                          df_main_dict['133'],
                          df_main_dict['134'],
                          df_main_dict['135'],
                          df_main_dict['136'],
                          df_main_dict['137'],
                         ], axis=0, join='outer').reset_index(drop=True)

df_trend_IF = pm2.pm2io.convert_long_dataframe_if(
    df_trend_all,
    coords_cols=coords_cols,
    #add_coords_cols=add_coords_cols,
    coords_defaults=coords_defaults,
    coords_terminologies=coords_terminologies,
    coords_value_mapping=coords_value_mapping['trend'],
    #coords_value_filling=coords_value_filling,
    filter_remove=filter_remove,
    #filter_keep=filter_keep,
    meta_data=meta_data,
    convert_str=True,
    time_format="%Y",
    )
    
df_trend_IF
       

,source,scenario (PRIMAP),provenance,area (ISO3),entity,unit,category (IPCC1996_2006_GIN_Inv),1990,1995,2000,2005,2010,2015,2018,2019
0,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,0,65.202,93.368,119.981,152.272,196.057,253.025,296.416,312.034
1,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1,6.465,7.066,6.489,5.984,4.849,5.360,5.931,5.866
2,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A,6.465,7.066,6.489,5.984,4.849,5.360,5.931,5.866
3,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A.1,0.032,0.027,0.024,0.020,0.016,0.002,0.005,0.001
4,GIN-GHG-Inventory,BUR1,measured,GIN,CH4,Gg CH4 / yr,1.A.2,0.006,0.012,0.018,0.023,0.028,0.024,0.026,0.033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,M.BK,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
540,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,M.BK.A,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
541,GIN-GHG-Inventory,BUR1,measured,GIN,SO2,Gg SO2 / yr,M.BK.M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "N2O") , "2019"]
df_trend_IF['entity'].unique()

array(['CH4', 'CO', 'CO2', 'N2O', 'NMVOC', 'NOx', 'SO2'], dtype=object)

In [28]:
# values in main table are assumed to be correct
#df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.D") & (df_trend_IF["entity"] == "CO2") , "2019"] = 0
#df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.D.1") & (df_trend_IF["entity"] == "CO2") , "2019"] = np.nan

# CH4 - values in main table are assumed to be correct
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "CH4") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "CH4") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "CH4") , "2000"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "CH4") , "2000"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "CH4") , "2010"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "CH4") , "2010"] = np.nan

# CO - values in main table are assumed to be correct
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "1.A.2") & (df_trend_IF["entity"] == "CO") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "CO") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "CO") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "CO") , "2000"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "CO") , "2000"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "CO") , "2010"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "CO") , "2010"] = np.nan

# N2O - values in main table are assumed to be correct
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "1.A.2") & (df_trend_IF["entity"] == "N2O") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "N2O") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "N2O") , "1990"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "N2O") , "2000"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "N2O") , "2000"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "N2O") , "2010"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "N2O") , "2010"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK") & (df_trend_IF["entity"] == "N2O") , "2019"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "M.BK.A") & (df_trend_IF["entity"] == "N2O") , "2019"] = np.nan

# NOx - values in main table are assumed to be correct
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.C") & (df_trend_IF["entity"] == "NOx") , "2019"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "3.C.1") & (df_trend_IF["entity"] == "NOx") , "2019"] = np.nan
df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == "3") & (df_trend_IF["entity"] == "NOx") , "2019"] = np.nan

# NMVOC - values in main table are assumed to be correct
entity = 'NMVOC'
for category, year in [
    ('0', '2000'),
    ('1', '2000'),
    ('1.A','2000'),
    ('1.A.1','2000'),
    ('1.A.2','2000'),
    ('1.A.3','2000'),
    ('1.A.4','2000'),
    ('2','2000'),
    ('2.H', '2000'),
    ('2.H.2', '2000'),
    ('0','2010'),
    ('1','2010'),
    ('1.A','2010'),
    ('1.A.1','2010'),
    ('1.A.2','2010'),
    ('1.A.3','2010'),
    ('1.A.4','2010'),
    ('2','2010'),
]:
    df_trend_IF.loc[(df_trend_IF["category (IPCC1996_2006_GIN_Inv)"] == category) & (df_trend_IF["entity"] == entity) , year] = np.nan


In [29]:
df_trend_IF['entity'].unique()

array(['CH4', 'CO', 'CO2', 'N2O', 'NMVOC', 'NOx', 'SO2'], dtype=object)

In [30]:
### convert to primap2 format ###
data_pm2_trend = pm2.pm2io.from_interchange_format(df_trend_IF)

2024-03-29 12:30:08.695 | DEBUG    | primap2.pm2io._interchange_format:from_interchange_format:320 - Expected array shapes: [[1, 1, 1, 1, 7, 78], [1, 1, 1, 1, 7, 78], [1, 1, 1, 1, 7, 78], [1, 1, 1, 1, 7, 78], [1, 1, 1, 1, 7, 78], [1, 1, 1, 1, 7, 78], [1, 1, 1, 1, 7, 78]], resulting in size 3,822.
/Users/danielbusch/Documents/UNFCCC_non-AnnexI_data/venv/lib/python3.10/site-packages/primap2/_data_format.py:481: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  all_dims = set(ds.dims.keys())
2024-03-29 12:30:08.961 | INFO     | primap2._data_format:ensure_valid_attributes:292 - Reference information is not a DOI: 'https://unfccc.int/BURs'


# Combine tables and save to IF and native format

In [31]:
#### combine


# discrepancies larger than 0.86 for area (ISO3)=GIN,
# category (IPCC1996_2006_GIN_Inv)=1.A.2, entity=NMVOC
# 1990-01-01  0.800000, 2000-01-01  0.800000, 2010-01-01  0.869848
# and
# (ISO3)=GIN, category (IPCC1996_2006_GIN_Inv)=1.A.2, time=1262304000000000000 (2019?)
# The values in the table are different / one is wrong
# merge main and energy
data_pm2 = data_pm2_main.pr.merge(data_pm2_energy,tolerance=1)

2024-03-29 12:30:13.712 | DEBUG    | primap2._merge:merge:230 - merging for NMVOC
2024-03-29 12:30:14.118 | DEBUG    | primap2._merge:merge:230 - merging for N2O
2024-03-29 12:30:14.408 | DEBUG    | primap2._merge:merge:230 - merging for CH4
2024-03-29 12:30:14.701 | DEBUG    | primap2._merge:merge:230 - merging for CO2
2024-03-29 12:30:14.993 | DEBUG    | primap2._merge:merge:230 - merging for SO2
2024-03-29 12:30:15.176 | DEBUG    | primap2._merge:merge:230 - merging for CO
2024-03-29 12:30:15.468 | DEBUG    | primap2._merge:merge:230 - merging for NOx


In [32]:
# merge lulucf 
data_pm2 = data_pm2.pr.merge(data_pm2_lulucf,tolerance=0.11)

2024-03-29 12:30:18.578 | DEBUG    | primap2._merge:merge:230 - merging for NMVOC
2024-03-29 12:30:19.081 | DEBUG    | primap2._merge:merge:230 - merging for N2O
2024-03-29 12:30:19.369 | DEBUG    | primap2._merge:merge:230 - merging for CH4
2024-03-29 12:30:19.802 | DEBUG    | primap2._merge:merge:230 - merging for CO2
2024-03-29 12:30:20.230 | DEBUG    | primap2._merge:merge:230 - merging for CO
2024-03-29 12:30:20.483 | DEBUG    | primap2._merge:merge:230 - merging for NOx


In [33]:
# merge waste 
# increasing tolerance to merge values for 4.C, 1990, N2O - 0.003 in sector table, 0.0034 in main table
data_pm2 = data_pm2.pr.merge(data_pm2_waste,tolerance=0.15)

2024-03-29 12:30:23.522 | DEBUG    | primap2._merge:merge:230 - merging for NMVOC
2024-03-29 12:30:23.995 | DEBUG    | primap2._merge:merge:230 - merging for N2O
2024-03-29 12:30:24.443 | DEBUG    | primap2._merge:merge:230 - merging for CH4
2024-03-29 12:30:24.730 | DEBUG    | primap2._merge:merge:230 - merging for CO2
2024-03-29 12:30:25.020 | DEBUG    | primap2._merge:merge:230 - merging for SO2
2024-03-29 12:30:25.289 | DEBUG    | primap2._merge:merge:230 - merging for CO
2024-03-29 12:30:25.552 | DEBUG    | primap2._merge:merge:230 - merging for NOx


In [34]:
data_pm2 = data_pm2.pr.merge(data_pm2_trend,tolerance=0.11)

2024-03-29 12:30:27.515 | DEBUG    | primap2._merge:merge:230 - merging for NMVOC
2024-03-29 12:30:27.636 | DEBUG    | primap2._merge:merge:230 - merging for N2O
2024-03-29 12:30:27.727 | DEBUG    | primap2._merge:merge:230 - merging for CH4
2024-03-29 12:30:27.818 | DEBUG    | primap2._merge:merge:230 - merging for CO2
2024-03-29 12:30:27.909 | DEBUG    | primap2._merge:merge:230 - merging for SO2
2024-03-29 12:30:27.998 | DEBUG    | primap2._merge:merge:230 - merging for CO
2024-03-29 12:30:28.035 | DEBUG    | primap2._merge:merge:230 - merging for NOx


In [35]:
# convert back to IF to have units in the fixed format ( per year / per a / per annum)
data_if = data_pm2.pr.to_interchange_format()

# ###
# save data to IF and native format
# ###
#pm2.pm2io.write_interchange_format(
#    output_folder / (output_filename + coords_terminologies["category"] + "_raw"), data_if)

#encoding = {var: compression for var in data_pm2.data_vars}
#data_pm2.pr.to_netcdf(
#    output_folder / (output_filename + coords_terminologies["category"] + "_raw.nc"),
#    encoding=encoding)

In [44]:
#for i in pd.Series(data_if['category (IPCC1996_2006_GIN_Inv)'].unique()).sort_values():
#    print(i)
#data_if.loc[(data_if["category (IPCC1996_2006_GIN_Inv)"] == '3.D.2')]
data_if['category (IPCC1996_2006_GIN_Inv)'].unique()

array(['0', '1', '1.A', '1.A.1', '1.A.1.a', '1.A.1.a.i', '1.A.2',
       '1.A.2.c', '1.A.2.e', '1.A.2.f', '1.A.2.i', '1.A.2.k', '1.A.2.m',
       '1.A.3', '1.A.3.a', '1.A.3.a.i', '1.A.3.a.ii', '1.A.3.b',
       '1.A.3.c', '1.A.3.d', '1.A.3.d.ii', '1.A.4', '1.A.4.b', '1.A.4.c',
       '1.A.4.c.i', '1.A.5', '2', '2.A', '2.C', '2.C.2', '2.D', '2.D.4',
       '2.F', '2.H', '2.H.2', '3', '3.A', '3.A.1', '3.A.1.a', '3.A.1.a.i',
       '3.A.1.a.ii', '3.A.1.c', '3.A.1.d', '3.A.1.h', '3.A.2', '3.A.2.a',
       '3.A.2.a.i', '3.A.2.a.ii', '3.A.2.c', '3.A.2.d', '3.A.2.h',
       '3.A.2.i', '3.A.2.j', '3.B', '3.B.1', '3.B.1.b', '3.B.2',
       '3.B.2.b', '3.B.3', '3.B.3.b', '3.B.4', '3.B.4.a', '3.B.4.b',
       '3.B.5', '3.B.5.b', '3.B.6', '3.B.6.b', '3.C', '3.C.1', '3.C.1.c',
       '3.C.7', '3.C.8', '3.D', '3.D.2', '4', '4.A', '4.B', '4.C',
       '4.C.2', '4.D', '4.D.1', '4.D.2', '4.E', 'M.BK', 'M.BK.A',
       'M.MULTIOP', '1.A.3.e', '2.A.1', '2.D.1', '2.F.1', '3.B.1.a',
       '3.B.1.b.i', '3.

In [45]:
data_proc_pm2 = data_pm2

country_processing_step1 = {
    'aggregate_cats': {
        'M.3.C.AG': {'sources': ['3.C.1', '3.C.2', '3.C.3', '3.C.4', '3.C.5',
                                 '3.C.6', '3.C.7', '3.C.8'],
                     'name': 'Aggregate sources and non-CO2 emissions sources on land '
                             '(Agriculture)'},
        'M.3.D.AG': {'sources': ['3.D.2'],
                     'name': 'Other (Agriculture)'},
        'M.AG.ELV': {'sources': ['M.3.C.AG', 'M.3.D.AG'],
                     'name': 'Agriculture excluding livestock'},
        'M.AG': {'sources': ['3.A', 'M.AG.ELV'],
                     'name': 'Agriculture'},
        'M.3.D.LU': {'sources': ['3.D.1'],
                     'name': 'Other (LULUCF)'},
        'M.LULUCF': {'sources': ['3.B', 'M.3.D.LU'],
                     'name': 'LULUCF'},
        'M.0.EL': {'sources': ['1', '2', 'M.AG', '4'],
                     'name': 'National total emissions excluding LULUCF'},
    },
    'basket_copy': {
        'GWPs_to_add': ["SARGWP100", "AR5GWP100", "AR6GWP100"],
        'entities': ["HFCS", "PFCS"],
        'source_GWP': gwp_to_use,
    },
}

gas_baskets = {
    'FGASES (SARGWP100)': ['HFCS (SARGWP100)', 'PFCS (SARGWP100)', 'SF6', 'NF3'],
    'FGASES (AR4GWP100)': ['HFCS (AR4GWP100)', 'PFCS (AR4GWP100)', 'SF6', 'NF3'],
    'FGASES (AR5GWP100)':['HFCS (AR5GWP100)', 'PFCS (AR5GWP100)', 'SF6', 'NF3'],
    'FGASES (AR6GWP100)':['HFCS (AR6GWP100)', 'PFCS (AR6GWP100)', 'SF6', 'NF3'],
    'KYOTOGHG (SARGWP100)': ['CO2', 'CH4', 'N2O', 'FGASES (SARGWP100)'],
    'KYOTOGHG (AR4GWP100)': ['CO2', 'CH4', 'N2O', 'FGASES (AR4GWP100)'],
    'KYOTOGHG (AR5GWP100)': ['CO2', 'CH4', 'N2O', 'FGASES (AR5GWP100)'],
    'KYOTOGHG (AR6GWP100)': ['CO2', 'CH4', 'N2O', 'FGASES (AR6GWP100)'],
}

# actual processing
#data_proc_pm2 = process_data_for_country(
#    data_proc_pm2,
#    gas_baskets=gas_baskets,
#    entities_to_ignore=[],
#    processing_info_country=country_processing_step1,
#)

In [46]:
import re
# All steps from process_data_for_country
# """
# Process data from DI interface (where necessary).
# * Downscaling including subtraction of time series
# * country specific sector aggregation
# * Conversion to IPCC2006 categories
# * general sector and gas basket aggregation (in new categories)
# """
entities_to_ignore=[],
processing_info_country=country_processing_step1,

# 0: gather information
data_country = data_proc_pm2
countries = list(data_country.coords[data_country.attrs["area"]].values)
if len(countries) > 1:
    raise ValueError(
        f"Found {len(countries)} countries. Only single country data "
        f"can be processed by this function. countries: {countries}"
    )
else:
    country_code = countries[0]

# get category terminology
cat_col = data_country.attrs["cat"]
temp = re.findall(r"\((.*)\)", cat_col)
cat_terminology_in = temp[0]

# get scenario
scenarios = list(data_country.coords[data_country.attrs["scen"]].values)
if len(scenarios) > 1:
    raise ValueError(
        f"Found {len(scenarios)} scenarios. Only single scenario data "
        f"can be processed by this function. Scenarios: {scenarios}"
    )
scenario = scenarios[0]

# get source
sources = list(data_country.coords["source"].values)
if len(sources) > 1:
    raise ValueError(
        f"Found {len(sources)} sources. Only single source data "
        f"can be processed by this function. Sources: {sources}"
    )
source = sources[0]

# check if category name column present
# TODO: replace 'name' in config by  'additional_cols' dict that defines the cols
#  and the values
if "orig_cat_name" in data_country.coords:
    cat_name_present = True
else:
    cat_name_present = False

In [47]:
# 1: general processing
# remove unused cats
data_country = data_country.dropna(f"category ({cat_terminology_in})", how="all")
# remove unused years
data_country = data_country.dropna(f"time", how="all")
# remove variables only containing nan
nan_vars_country = [
    var
    for var in data_country.data_vars
    if bool(data_country[var].isnull().all().data) is True
]
print(f"removing all-nan variables: {nan_vars_country}")
data_country = data_country.drop_vars(nan_vars_country)


removing all-nan variables: []


In [48]:
tolerance = 0.01
agg_tolerance = tolerance

aggregate_cats_current = country_processing_step1["aggregate_cats"]

print(
    f"Aggregating categories for country {country_code}, source {source}, "
    f"scenario {scenario}"
)
for cat_to_agg in aggregate_cats_current:
    print(f"Category: {cat_to_agg}")
    source_cats = aggregate_cats_current[cat_to_agg]["sources"]
    data_agg = data_country.pr.loc[{"category": source_cats}].pr.sum(
        dim="category", skipna=True, min_count=1
    )
    #data_agg = data_country.pr.loc[{"category": source_cats}]
    nan_vars = [
        var
        for var in data_agg.data_vars
        if data_agg[var].isnull().all().data is True
    ]
    data_agg = data_agg.drop(nan_vars)
    if len(data_agg.data_vars) > 0:
        data_agg = data_agg.expand_dims(
            [f"category (" f"{cat_terminology_in})"]
        )
        data_agg = data_agg.assign_coords(
            coords={
                f"category ({cat_terminology_in})": (
                    f"category ({cat_terminology_in})",
                    [cat_to_agg],
                )
            }
        )
        if cat_name_present:
            cat_name = aggregate_cats_current[cat_to_agg]["name"]
            data_agg = data_agg.assign_coords(
                coords={
                    "orig_cat_name": (
                        f"category ({cat_terminology_in})",
                        [cat_name],
                    )
                }
            )
        data_country = data_country.pr.merge(
            data_agg, tolerance=agg_tolerance
        )
    else:
        print(f"no data to aggregate category {cat_to_agg}")

Aggregating categories for country GIN, source GIN-GHG-Inventory, scenario BUR1
Category: M.3.C.AG


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/1732274552.py:22: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  data_agg = data_agg.drop(nan_vars)


Category: M.3.D.AG


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/1732274552.py:22: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  data_agg = data_agg.drop(nan_vars)


Category: M.AG.ELV


...

/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/1732274552.py:22: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  data_agg = data_agg.drop(nan_vars)


Category: M.LULUCF


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/1732274552.py:22: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  data_agg = data_agg.drop(nan_vars)


Category: M.0.EL


/var/folders/th/tt5f76gd0q139kzgw6b9m8l40000gn/T/ipykernel_10373/1732274552.py:22: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  data_agg = data_agg.drop(nan_vars)


In [51]:
from UNFCCC_GHG_data.helper import GWP_factors

In [52]:
# copy HFCs and PFCs with default factors
GWPs_to_add = country_processing_step1["basket_copy"]["GWPs_to_add"]
entities = country_processing_step1["basket_copy"]["entities"]
source_GWP = country_processing_step1["basket_copy"]["source_GWP"]
for entity in entities:
    data_source = data_country[f"{entity} ({source_GWP})"]
    for GWP in GWPs_to_add:
        data_GWP = (
            data_source * GWP_factors[f"{source_GWP}_to_{GWP}"][entity]
        )
        data_GWP.attrs["entity"] = entity
        data_GWP.attrs["gwp_context"] = GWP
        data_country[f"{entity} ({GWP})"] = data_GWP

In [54]:
import xarray as xr

In [55]:
# create gas baskets
entities_present = set(data_country.data_vars)
for basket in gas_baskets.keys():
    basket_contents_present = [
        gas for gas in gas_baskets[basket] if gas in entities_present
    ]
    if len(basket_contents_present) > 0:
        if basket in list(data_country.data_vars):
            data_country[basket] = data_country.pr.fill_na_gas_basket_from_contents(
                basket=basket,
                basket_contents=basket_contents_present,
                skipna=True,
                min_count=1,
            )
        else:
            try:
                # print(data_country.data_vars)
                data_country[basket] = xr.full_like(
                    data_country["CO2"], np.nan
                ).pr.quantify(units="Gg CO2 / year")
                data_country[basket].attrs = {
                    "entity": basket.split(" ")[0],
                    "gwp_context": basket.split(" ")[1][1:-1],
                }
                data_country[basket] = data_country.pr.gas_basket_contents_sum(
                    basket=basket,
                    basket_contents=basket_contents_present,
                    min_count=1,
                )
                entities_present.add(basket)
            except Exception as ex:
                print(
                    f"No gas basket created for {country_code}, {source}, "
                    f"{scenario}: {ex}"
                )

In [57]:
from datetime import date

In [59]:
# amend title and comment
data_country.attrs["comment"] = (
    data_country.attrs["comment"] + f" Processed on " f"{date.today()}"
)
data_country.attrs["title"] = (
    data_country.attrs["title"] + f" Processed on " f"{date.today()}"
)

data_proc_pm2 = data_country

In [ ]:
# ###
# save data to IF and native format
# ###
terminology_proc = coords_terminologies['category']

data_proc_if = data_proc_pm2.pr.to_interchange_format()
if not output_folder.exists():
    output_folder.mkdir()
pm2.pm2io.write_interchange_format(
    output_folder / (output_filename + terminology_proc), data_proc_if)

encoding = {var: compression for var in data_proc_pm2.data_vars}
data_proc_pm2.pr.to_netcdf(
    output_folder / (output_filename + terminology_proc + ".nc"),
    encoding=encoding)

# Annex (maybe not needed)

In [ ]:
#data_agg = data_agg.pr.sum(dim="category", skipna=True, min_count=1)
#test_if = data_agg
#test_if = test_if.pr.sum(dim="entity", skipna=True, min_count=1)
#test_if.pr.to_interchange_format()
#type(data_agg.pr.to_interchange_format())
data_agg.pr.to_interchange_format()
#data_agg.sum(dim="category (IPCC1996_2006_GIN_Inv)", skipna=True, min_count=1).pr.to_interchange_format()

In [ ]:
# category aggregation
# aggregate cats in process_data_for_country
# processing for removals not neccessary here
# - all sectors need to be present: we probably need sector M.AG (3.A, parts of 3.C and parts of M.A)
# aggregate categories, check Malaysia config, check table IPCC1996 / IPCC2006 category table 

data_proc_pm2 = data_pm2

# actual processing
data_proc_pm2 = process_data_for_country(
    data_proc_pm2,
    gas_baskets=gas_baskets,
    entities_to_ignore=[],
    processing_info_country=country_processing_step1,
)

'AND_BUR2': {
    # '3.A': {'sources': ['3.A.1', '3.A.2'], 'name': 'Livestock'},
    'M.3.C.1.AG': {'sources': ['3.C.1.b', '3.C.1.c'], 'name': 'Emissions from Biomass Burning (Agriculture)'},
    'M.3.C.1.LU': {'sources': ['3.C.1.a', '3.C.1.d'], 'name': 'Emissions from Biomass Burning (LULUCF)'},
    # '3.C.1': {'sources': ['M.3.C.1AG', 'M.3.C.1LU'], 'name': 'Emissions from Biomass Burning'},
    # '3.C': {'sources': ['3.C.1', '3.C.2', '3.C.3', '3.C.4', '3.C.5', '3.C.6', '3.C.7', '3.C.8'],
    #        'name': 'Aggregate sources and non-CO2 emissions sources on land'},
    'M.3.C.AG': {'sources': ['M.3.C.1.AG', '3.C.2', '3.C.3', '3.C.4', '3.C.5', '3.C.6', '3.C.7', '3.C.8'],
                 'name': 'Aggregate sources and non-CO2 emissions sources on land (Agriculture)'},
    'M.AG.ELV': {'sources': ['M.3.C.AG'], 'name': 'Agriculture excluding livestock emissions'},
    'M.AG': {'sources': ['3.A', 'M.AG.ELV'], 'name': 'Agriculture'},
    'M.LULUCF': {'sources': ['3.B', 'M.3.C.1.LU', '3.D.1'], 'name': 'Land Use, Land Use Change, and Forestry'},
    # '3': {'sources': ['M.AG', 'M.LULUCF'], 'name': 'AFOLU'},
    '0': {'sources': ['1', '2', '3', '4'], 'name': 'National Total'},
    'M.0.EL': {'sources': ['1', '2', 'M.AG', '4'], 'name': 'National Total Excluding LULUCF'},
},

# all 3.D, M.3.LU to M.LULUCF
# M.0.EL is missing in Guinea report

In [ ]:
# gas baskets
# docs im repo anschauen
# TODO:  downscaling

# 
data_proc_pm2 = process_data_for_country(
    data_proc_pm2,
    entities_to_ignore=[],
    gas_baskets=gas_baskets,
    processing_info_country=country_processing_step2, # maybe step 2 not necessary
    cat_terminology_out = terminology_proc,
    category_conversion = cat_conversion, # probably not
    sectors_out = sectors_to_save,
)

In [ ]:
# ###
# ## process the data
# ###
data_proc_pm2 = data_pm2

# combine CO2 emissions and removals
#data_proc_pm2["CO2"] = data_proc_pm2[["CO2 emissions", "CO2 removals"]].pr.sum\
#    (dim="entity", skipna=True, min_count=1)
#data_proc_pm2["CO2"].attrs['entity'] = 'CO2'

# aggregate gase
# basket copy: hfcs and pfcs
# convert from AR4, see example Malaysia config BUR4 (country processing, basket copy, gas baskets)

country_processing_step1 = { # not needed, will be done with gas baskets
    'aggregate_gases': {
        'KYOTOGHG': {
            'basket': 'KYOTOGHG (AR4GWP100)',
            'basket_contents': ['CO2', 'CH4', 'N2O', 'SF6',
                                'HFCS (AR4GWP100)', 'PFCS (AR4GWP100)'],
            'skipna': True,
            'min_count': 1,
            'sel': {f'category ({coords_terminologies["category"]})':
                [
                    '0', '1', '1.A', '1.A.1', '1.A.2', '1.A.3',
                    '1.A.4', '1.B', '1.B.1', '1.B.2',
                    '1.C',
                    '2', '2.A', '2.A.1', '2.A.2', '2.A.3', '2.A.4',
                    '2.B', '2.C', '2.D', '2.H',
                    '3', '3.A', '3.B', '3.C', '3.D', '3.E', '3.F', '3.G',
                    '3.H', '3.I',
                    '4', '4.A', '4.B', '4.C', '4.D', '4.E',
                    '5', '5.A', '5.B', '5.C', '5.D'
                ]
            }, # not tested
        },
    },
}

# actual processing
data_proc_pm2 = process_data_for_country(
    data_proc_pm2,
    entities_to_ignore=[],
    gas_baskets={},
    processing_info_country=country_processing_step1,
)

# check if all data is float


In [ ]:
data_proc_pm2 = process_data_for_country(
    data_proc_pm2,
    entities_to_ignore=[],
    gas_baskets=gas_baskets,
    processing_info_country=country_processing_step2,
    cat_terminology_out = terminology_proc,
    category_conversion = cat_conversion,
    sectors_out = sectors_to_save,
)

# adapt source and metadata
# TODO: processing info is present twice
current_source = data_proc_pm2.coords["source"].values[0]
data_temp = data_proc_pm2.pr.loc[{"source": current_source}]
data_proc_pm2 = data_proc_pm2.pr.set("source", 'BUR_NIR', data_temp)

# ###
# save data to IF and native format
# ###
data_proc_if = data_proc_pm2.pr.to_interchange_format()
if not output_folder.exists():
    output_folder.mkdir()
pm2.pm2io.write_interchange_format(
    output_folder / (output_filename + terminology_proc), data_proc_if)

encoding = {var: compression for var in data_proc_pm2.data_vars}
data_proc_pm2.pr.to_netcdf(
    output_folder / (output_filename + terminology_proc + ".nc"),
    encoding=encoding)